In [188]:
import pandas as pd
import numpy as np
import anndata as ad
import xarray as xr
from SpatialDE import spatial_patterns
import pickle

In [ ]:
### single-cell eQTL effects

In [85]:
filename = "/nfs/leia/research/stegle/acuomo/singlecell_endodiff/data/pipeline_snakemakes/output_structlmm_alldays_10PCs/eqtl_tot_genetic_effect_all.txt"
df = pd.read_csv(filename, sep = "\t", index_col=0)
df.head()

,ENSG00000176125_UFSP1-7_100492322_C_T,ENSG00000175106_TVP23C-17_15478507_G_C,ENSG00000147677_EIF3H-8_117688031_G_A,ENSG00000204389_HSPA1A-6_31761865_T_C,ENSG00000204389_HSPA1A-6_31785228_G_C,ENSG00000116731_PRDM2-1_14235022_G_A,ENSG00000203485_INF2-14_105216381_C_T,ENSG00000091317_CMTM6-3_32523777_T_C,ENSG00000126524_SBDS-7_66440718_A_G,ENSG00000144134_RABL2A-2_114406459_G_A,...,ENSG00000206535_LNP1-3_100055273_G_A,ENSG00000206535_LNP1-3_100060779_C_T,ENSG00000206535_LNP1-3_100181999_G_T,ENSG00000143971_ETAA1-2_67712034_T_G,ENSG00000054277_OPN3-1_241892235_T_C,ENSG00000069667_RORA-15_61467734_T_C,ENSG00000173894_CBX2-17_77814533_G_A,ENSG00000164930_FZD6-8_104312432_A_G,ENSG00000164930_FZD6-8_104317011_A_G,ENSG00000164930_FZD6-8_104345778_A_G
21843_1#99,-0.092291,-0.020901,-0.054118,-0.617919,-0.654549,-0.013270,-0.185045,0.150489,-0.064830,0.041793,...,-0.039099,-0.142666,-0.091894,-0.094953,-0.080701,-0.067996,0.026881,-0.202234,-0.197769,-0.203371
24229_5#47,-0.094179,-0.418200,0.039307,-1.340217,-1.205610,-0.028206,-0.306860,0.050959,-0.089726,0.208231,...,-0.061028,-0.100131,-0.267392,-0.111616,-0.015448,-0.110905,0.030413,-0.006998,0.006067,-0.068799
25535_2#136,-0.080449,-0.044425,-0.047899,-0.879945,-0.889774,0.008512,-0.163487,0.146643,-0.030945,0.042243,...,-0.106094,-0.148331,-0.133007,-0.088487,-0.088059,-0.050601,0.009392,-0.249793,-0.244262,-0.224866
23794_1#233,-0.091711,-0.042417,-0.120662,-0.429285,-0.449291,-0.049670,-0.173315,0.185699,-0.005271,0.036158,...,-0.139418,-0.191834,-0.090162,-0.129926,-0.137495,-0.043284,0.043521,-0.317619,-0.326827,-0.264026
25475_3#27,-0.089183,-0.029810,-0.100041,-0.405588,-0.401307,-0.033448,-0.225870,0.170078,-0.069707,0.007676,...,-0.225352,-0.196503,-0.158505,-0.112523,-0.072948,-0.041999,0.088213,-0.265396,-0.300596,-0.242712


In [ ]:
### PC coordinates

In [104]:
pcs_filename = "/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_endodiff/10pcs_500hvgs.csv"
df_pcs = pd.read_csv(pcs_filename, index_col=0)
df_pcs.head()
df_pcs.index

Index(['21843_1#10', '21843_1#100', '21843_1#101', '21843_1#102',
       '21843_1#103', '21843_1#105', '21843_1#106', '21843_1#107',
       '21843_1#108', '21843_1#109',
       ...
       '24539_8#88', '24539_8#89', '24539_8#90', '24539_8#91', '24539_8#92',
       '24539_8#93', '24539_8#94', '24539_8#95', '24539_8#97', '24539_8#98'],
      dtype='object', length=36044)

In [105]:
### build "fake" AnnData file
#
# counts: single-cell eQTL effects
# adata.obsm['spatial']: coordinates (PCA, UMAP..)

In [136]:
cells = df_pcs.index.values
cells = sorted(set(list(df.index.values)).intersection(cells))

In [137]:
df = df[df.index.isin(cells)]
df.shape

(33478, 4062)

In [138]:
df_pcs = df_pcs[df_pcs.index.isin(cells)]
df_pcs.shape

(33478, 10)

In [153]:
df = df.sort_index(axis=0)
df_pcs = df_pcs.sort_index(axis=0)

In [154]:
obs_names = df.index.values
obs = pd.DataFrame(index=obs_names)
# obs
n_obs = len(obs)
n_obs

33478

In [155]:
var_names = df.columns.values
var = pd.DataFrame(index=var_names)
# var
n_vars = len(var)
n_vars

4062

In [156]:
X = df.to_numpy()
X.shape
type(X)

numpy.ndarray

In [180]:
adata = ad.AnnData(X=X, obs=obs, var=var, dtype='float')
adata

AnnData object with n_obs × n_vars = 33478 × 4062

In [181]:
df_pcs.head()

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10
21241_6#10,32.193582,-2.686568,5.318807,-17.205658,10.304770,-5.122648,-10.114578,0.475732,9.525488,3.975142
21241_6#100,29.271910,-3.382487,7.217431,-26.379431,2.138624,-3.758808,-9.407537,1.431489,5.081567,3.704345
21241_6#101,30.458309,3.273150,14.441991,-6.860621,-12.636434,-6.208608,-7.710519,7.424244,-4.576510,7.680715
21241_6#103,31.063225,9.701125,12.602948,-9.719614,-12.252897,-5.496166,-2.239845,3.811890,0.578312,3.418790
21241_6#104,36.168878,1.966633,1.364692,-25.534776,0.097970,-0.272529,-5.594344,5.138253,7.231300,0.642229


In [182]:
adata.obsm['spatial'] = df_pcs
# adata.obsm['spatial'] = df_pcs.to_numpy()

In [183]:
# adata
adata.obsm['spatial'].shape

(33478, 10)

In [189]:
adata

AnnData object with n_obs × n_vars = 33478 × 4062
    obsm: 'spatial'

In [190]:
adata.write("/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_endodiff/new/effect_sizes_clustering/non_scaled_old_eff_sizes_10pcs_as_spatial.h5")

In [184]:
### run SpatialDE to identify clusters

In [ ]:
sp = spatial_patterns(adata, normalized = True)

In [ ]:
sp

In [ ]:
pickle.dump(sp, "/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_endodiff/new/effect_sizes_clustering/non_scaled_old_eff_sizes_10pcs_spatial_patterns.p")

In [ ]:
adata.write("/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_endodiff/new/effect_sizes_clustering/non_scaled_old_eff_sizes_10pcs_as_spatial.h5")